In [1]:
#!pip install dspy-ai==2.1.9 weaviate-client==3.26.2 > /dev/null

In [2]:
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune
import os
from dotenv import load_dotenv
import pathlib
import re
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score
import dsp
import numpy as np
from scipy import sparse
import json
import os
#from dspy.utils import deduplicate

In [3]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_distances

In [4]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [5]:
path_env = pathlib.Path(os.getcwd()).parent.parent / '.env'
print(path_env)
load_dotenv(path_env)
api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = api_key

/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/.env


In [6]:
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join('.', 'cache')

In [7]:
llm = dspy.OpenAI(
    model="gpt-4o-2024-05-13" , #"gpt-4-0125-preview",  #gpt-4o-2024-05-13, #"gpt-4-1106-preview", # TODO: try turbo-instruct,
    max_tokens=1000)

# Assumes the Weaviate collection has a text key `content`
dspy.settings.configure(lm=llm)

### Defining the training data

In [8]:
import pandas as pd
df = pd.read_excel("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/src/contradictions/contrad_ex.xlsx")
df = df.astype(str)
df

,ID,Context,Text,Rationale,LabelInt,LabelStr,Faithfulness
0,0,"Testosterone is a hormone that your gonads (sex organs) mainly produce. More specifically, the testicles in people assigned male at birth (AMAB) and the ovaries in people assigned female at birth (AFAB) produce testosterone.","In the brain, the hypothalamus releases chemicals (hormones) that cause the pituitary gland to release hormones called gonadotropins. Gonadotropins stimulate the growth of the sex glands (also called gonads), which in boys are the testicles, and in girls are the ovaries. In boys, the testicles release testosterone; in girls, the ovaries release estrogen. Puberty usually starts between the ages of 8 and 13 in girls and 9 and 14 in boys.","The context highlights that ovaries produce testosterone without detailing its relatively small amount compared to estrogen, while the new text emphasizes estrogen as the primary hormone from ovaries, possibly implying that testosterone is not produced by ovaries at all.",3,Minor Discrepancy,False
1,1,"With one important exception. The exception was the shot. Compared to women using a hormonal IUD, women using the shot were three times more likely to have a blood clot.\t","Careful, not all methods are safe for new moms to use: As if they didn’t have their hands full already, women who have just given birth have to be especially careful when choosing a contraceptive. The CDC has just updated its guidelines on which methods are safe for new moms to use, and when. While hormonal methods that don’t contain estrogen—the shot, the Mirena IUD, the implant and the mini-pill—are safe for women to use immediately after giving birth, combination pills, which contain estrogen, shouldn’t be used until three weeks after giving birth, since estrogen can increase a woman’s risk of blood clots.\t","The context notes an increased risk of blood clots with the contraceptive shot compared to hormonal IUDs, the new text emphasizes its safety postpartum due to the absence of estrogen, despite the known clotting risk.",3,Minor Discrepancy,False
2,2,"Cal’s skin was described to be jaundiced again through visual assessment when he was 23 hours old, but a bilirubin test was not done. Neither was his blood typed or a Coombes test performed.\t","Diagnosis: At a minimum, babies should be checked for jaundice every 8 to 12 hours in the first 48 hours of life. It is important for your baby to be seen by a nurse or doctor when the baby is between 3 and 5 days old, because this is usually when a baby’s bilirubin level is highest. This is why, if your baby is discharged before age 72 hours, your baby should be seen within 2 days of discharge. The timing of this visit may vary depending on your baby’s age when released from the hospital and other factors.\t","The context describes a case where a baby showing signs of jaundice was not given a bilirubin test or other relevant tests before being discharged at 36 hours old, which contradicts the guidelines in the new text that recommend checking for jaundice every 8 to 12 hours in the first 48 hours and ensuring follow-up visits if discharged before 72 hours.\t",2,Major Discrepancy,False
3,3,The word prenatal means before birth. Trimester means 3 months. A normal pregnancy is around 10 months and has 3 trimesters.\t,"The average length of pregnancy is 280 days, or 40 weeks. But there is no way to know exactly when you will go into labor. Most women give birth between 38 and 41 weeks of pregnancy. The more you know about what to expect during labor, the better prepared you will be once it begins.\t","The context states that a normal pregnancy is around 10 months (which would imply 43-44 weeks), while the new text states that the average length of pregnancy is 40 weeks, with most women giving birth between 38 and 41 weeks.\t",1,Direct Contradiction,False
4,4,Key points about pregnancy loss: Pregnancy loss is the death of an unborn baby (fetus) at any time during pregnancy. Pregnancy loss occurs in up

In [9]:
data = []
for i, row in df.iterrows():
    data.append({
        "id": row["ID"],
        "context": row["Context"],
        "text": row["Text"],
        "rationale": row["Rationale"],
        "answer": row["LabelInt"],
        "LabelStr": row["LabelStr"],
        "Faithfulness": row["Faithfulness"]
    })
    

# Create training examples
trainset = [
    dspy.Example({**row}).with_inputs('context', 'text') for row in data[:4]
]

devset = [
    dspy.Example({**row}).with_inputs('context', 'text') for row in data[4:]
]

In [10]:
# Define the CheckContradictions signature
class CheckContradictions(dspy.Signature):
    """Identify if there are contradictions with the provided context."""
    context = dspy.InputField(desc="facts here are assumed to be true")
    text = dspy.InputField()
    Faithfulness = dspy.OutputField(desc="True/False indicating if text contradicts the context")
    rationale = dspy.OutputField(desc="Why the new text contradicts the context or not")
    
class CheckContradictionType(dspy.Signature):
    """Identify the type of contradiction detected"""
    context = dspy.InputField(desc="facts here are assumed to be true")
    text = dspy.InputField()
    rationale = dspy.OutputField(desc="Why the new text contradicts the context or not")
    LabelInt = dspy.OutputField(desc="A rating between 0 and 4: 0 = No Contradiction, 1 = Direct Contradiction, 2 = Major Discrepancy, 3 = Minor Discrepancy")

In [11]:
class RuleFewShot(dspy.Module):
    def __init__(self):
        super().__init__()
        self.check_contradictions = dspy.ChainOfThought(CheckContradictions)

    def forward(self, context, text):
        prediction = self.check_contradictions(context=context, text=text)
        return dspy.Prediction(context=context, text=text, Faithfulness=prediction.Faithfulness, rationale=prediction.rationale)
    
class RuleFewShotType(dspy.Module):
    def __init__(self):
        super().__init__()
        self.check_contradictions = dspy.ChainOfThought(CheckContradictionType)

    def forward(self, context, text):
        prediction = self.check_contradictions(context=context, text=text)
        return dspy.Prediction(context=context, text=text, rationale=prediction.rationale, answer=prediction.LabelInt)
    
# RAG class
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.check_contradictions = dspy.ChainOfThought(CheckContradictions)
    
    def forward(self, context, text):
        retrieved_passages = self.retrieve.forward(text).passages
        prediction = self.check_contradictions(context=retrieved_passages, text=text)
        return dspy.Prediction(context=retrieved_passages, answer=prediction.faithfulness)

In [12]:
contradiction_checker = RuleFewShotType()
output = contradiction_checker(context=devset[3].context, text=devset[3].text)
llm.inspect_history(1)





Identify the type of contradiction detected

---

Follow the following format.

Context: facts here are assumed to be true

Text: ${text}

Reasoning: Let's think step by step in order to ${produce the LabelInt}. We ...

Label Int: A rating between 0 and 4: 0 = No Contradiction, 1 = Direct Contradiction, 2 = Major Discrepancy, 3 = Minor Discrepancy

---

Context: Regular physical activity is essential for maintaining a healthy weight.

Text: Weight management is primarily influenced by diet, not physical activity, according to recent studies.

Reasoning: Let's think step by step in order to Reasoning: Let's think step by step in order to identify the type of contradiction. We start with the context, which states that regular physical activity is essential for maintaining a healthy weight. The text, however, claims that weight management is primarily influenced by diet, not physical activity, according to recent studies. This presents a direct contradiction to the context, as it down

In [96]:
# Define the CheckContradictions signature
class Divider(dspy.Signature):
    """Extract common entities between the given sentences
    
    Example:
    SENTENCE1 = Pregnancy duration is 10 months
    SENTENCE2 = The average lenght of pregnancy is 12 weeks
    
    ENTITIES1 = "pregnancy_duration": "10 months",
    ENTITIES2 = "average_pregnancy_length": "12 weeks"
    """
    
    SENTENCE1 = dspy.InputField()
    SENTENCE2 = dspy.InputField()
    
    ENTITIES1 = dspy.OutputField(desc="Entities from the first sentence")
    ENTITIES2 = dspy.OutputField(desc="Entities from the second sentence")
    
    
class DividerModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.divider = dspy.ChainOfThought(Divider)
    
    def forward(self, sentence1, sentence2):
        prediction = self.divider(SENTENCE1=sentence1, SENTENCE2=sentence2)
        return dspy.Prediction(ent1=prediction.ENTITIES1, ent2=prediction.ENTITIES2)
    
divider = DividerModule()
output = divider(sentence1="Pregnancy duration is 10 months", sentence2="The average length of pregnancy is 12 weeks")
llm.inspect_history(1)





Extract common entities between the given sentences
    
    Example:
    SENTENCE1 = Pregnancy duration is 10 months
    SENTENCE2 = The average lenght of pregnancy is 12 weeks
    
    ENTITIES1 = "pregnancy_duration": "10 months",
    ENTITIES2 = "average_pregnancy_length": "12 weeks"

---

Follow the following format.

SENTENCE 1: ${SENTENCE1}

SENTENCE 2: ${SENTENCE2}

ENTITIES 1: Entities from the first sentence

Reasoning: Let's think step by step in order to ${produce the ENTITIES2}. We ...

ENTITIES 2: Entities from the second sentence

---

SENTENCE 1: Pregnancy duration is 10 months

SENTENCE 2: The average length of pregnancy is 12 weeks

ENTITIES 1: ENTITIES 1: "pregnancy_duration": "10 months"

Reasoning: Let's think step by step in order to produce the ENTITIES2. We need to identify the key information in the second sentence that corresponds to the entities in the first sentence. The second sentence mentions "average length of pregnancy" and provides a value of "12 w

In [174]:
# Define the CheckContradictions signature
class Divider(dspy.Signature):
    """if you were using this text and JUST this text to answer questions, which questions could you answer
    
    Examples
    --------
    
    
    """
    
    CONTEXT = dspy.InputField(desc="Context to generate questions from")
    QUESTIONS = dspy.OutputField(desc="Questions extracted from the context and their reponses")    
    
class DividerModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.divider = dspy.ChainOfThought(Divider)
    
    def forward(self, context):
        prediction = self.divider(CONTEXT=context)
        return dspy.Prediction(q=prediction.QUESTIONS)
    
divider = DividerModule()
output = divider(context="chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge")
llm.inspect_history(1)





if you were using this text and JUST this text to answer questions, which questions could you answer

---

Follow the following format.

CONTEXT: Context to generate questions from
Reasoning: Let's think step by step in order to ${produce the QUESTIONS}. We ...
QUESTIONS: Questions extracted from the context and their reponses

---

CONTEXT: chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge
Reasoning: Let's think step by step in order to CONTEXT: chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge
Reasoning: Let's think step by step in order to produce the QUESTIONS. We need to identify the key elements and symptoms 

In [175]:
for el in retrieve_similar_documents(doc.text, model_en, index_en, ids_en, texts_en, k=20):
    print(divider(context=el["text"]))

Prediction(
    q='1. What is chorioamnionitis?\n   - Chorioamnionitis is a condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.\n\n2. What are the symptoms of chorioamnionitis?\n   - The symptoms of chorioamnionitis include unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.\n\n3. Can chorioamnionitis affect the heart rate of the fetus?\n   - Yes, chorioamnionitis can cause a rapid heart rate in the fetus.\n\n4. Can chorioamnionitis cause a high white blood cell count?\n   - Yes, chorioamnionitis can cause a high white blood cell count.\n\n5. What type of discharge might indicate chorioamnionitis?\n   - Foul-smelling vaginal discharge might indicate chorioamnionitis.\n\n6. Can chorioamnionitis cause a

In [101]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to extract entities from sentences
def extract_entities(sentence1, sentence2):
    # Process the sentences with spaCy
    doc1 = nlp(sentence1)
    doc2 = nlp(sentence2)
    
    # Extract entities
    entities1 = {ent.label_: ent.text for ent in doc1.ents}
    entities2 = {ent.label_: ent.text for ent in doc2.ents}
    
    return entities1, entities2

# Example sentences
sentence1 = "Pregnancy duration is 10 months"
sentence2 = "The average length of pregnancy is 12 weeks"

# Extract entities
entities1, entities2 = extract_entities(sentence1, sentence2)

# Print the results
print("Entities from Sentence 1:", entities1)
print("Entities from Sentence 2:", entities2)

Entities from Sentence 1: {'DATE': '10 months'}
Entities from Sentence 2: {'DATE': '12 weeks'}


In [102]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to extract nouns from sentences
def extract_nouns(sentence1, sentence2):
    # Process the sentences with spaCy
    doc1 = nlp(sentence1)
    doc2 = nlp(sentence2)
    
    # Extract nouns
    nouns1 = [token.text for token in doc1 if token.pos_ == "NOUN"]
    nouns2 = [token.text for token in doc2 if token.pos_ == "NOUN"]
    
    return nouns1, nouns2

# Example sentences
sentence1 = "Pregnancy duration is 10 months"
sentence2 = "The average length of pregnancy is 12 weeks"

# Extract nouns
nouns1, nouns2 = extract_nouns(sentence1, sentence2)

# Print the results
print("Nouns from Sentence 1:", nouns1)
print("Nouns from Sentence 2:", nouns2)


Nouns from Sentence 1: ['Pregnancy', 'duration', 'months']
Nouns from Sentence 2: ['length', 'pregnancy', 'weeks']


In [13]:
class CombinedCheckContradictions(dspy.Module):
    def __init__(self):
        super().__init__()
        self.check_contradictions = dspy.ChainOfThought(CheckContradictions)
        self.check_contradiction_type = dspy.ChainOfThought(CheckContradictionType)
        
    def process_faithfulness(self, faithfulness_resp):
        if "True" in faithfulness_resp:
            return True
        elif "False" in faithfulness_resp:
            return False
        else:
            print("Error parsing response of faithfulness")
    
    def process_labelint(self, label_int_resp):
        if "3" in dsp.normalize_text(label_int_resp):
            clean_pred = "3"
        elif "2" in dsp.normalize_text(label_int_resp):
            clean_pred = "2"
        elif "1" in dsp.normalize_text(label_int_resp):
            clean_pred = "1"
        elif "0" in dsp.normalize_text(label_int_resp):
            clean_pred = "0"
        else:
            clean_pred = "-1"
        return clean_pred
            
    def forward(self, context, text):
        # First check for contradictions
        contradiction_result = self.check_contradictions(context=context, text=text)
        faithfulness = self.process_faithfulness(contradiction_result.Faithfulness)
        rationale_contradiction = contradiction_result.rationale
        
        
        # If a contradiction is detected, check the type of contradiction
        if not faithfulness:
            type_result = self.check_contradiction_type(context=context, text=text)
            rationale_type = type_result.rationale
            label_int = self.process_labelint(type_result.LabelInt)
        else:
            rationale_type = rationale_contradiction
            label_int = str(0) # No contradiction

        # Combine the results
        combined_rationale = f"{rationale_contradiction}. {rationale_type}"
        
        return dspy.Prediction(
            Faithfulness=faithfulness,
            rationale=combined_rationale,
            answer=label_int
        )

contradiction_checker = CombinedCheckContradictions()
output = contradiction_checker(context=devset[0].context, text=devset[0].text)
output

Prediction(
    Faithfulness=False,
    rationale='The text states that "Around 15% of pregnancies end in miscarriage," which contradicts the context that says "Pregnancy loss occurs in up to 1 in every 4 pregnancies" (which is 25%). Additionally, the text mentions that "everyone’s risk of miscarriage declines each week of pregnancy if the pregnant person has no other health conditions," which is not supported by the context provided. The context does not specify that the risk declines each week or that it is contingent on the absence of other health conditions.. Reasoning: Let\'s think step by step in order to identify any contradictions or discrepancies. \n\n1. The context states that pregnancy loss occurs in up to 1 in every 4 pregnancies, which translates to a 25% risk.\n2. The text mentions that around 15% of pregnancies end in miscarriage, which is lower than the 25% mentioned in the context.\n3. The context states that most pregnancy losses happen during the first trimester, whi

In [14]:
def validate_answer_contained(example, pred, trace=None):
    """
    if "3" in dsp.normalize_text(pred.answer):
        clean_pred = "3"
    elif "2" in dsp.normalize_text(pred.answer):
        clean_pred = "2"
    elif "1" in dsp.normalize_text(pred.answer):
        clean_pred = "1"
    elif "0" in dsp.normalize_text(pred.answer):
        clean_pred = "0"
    else:
        clean_pred = "-1"
    """
    print(pred.answer, example.answer, pred.answer == example.answer)
    return pred.answer == example.answer


In [15]:
teleprompter = BootstrapFewShotWithRandomSearch(
    metric=validate_answer_contained,
    max_bootstrapped_demos=4,
    max_labeled_demos=16,
    num_candidate_programs=16,
    max_rounds=1,
)

compiled_classifier = teleprompter.compile(CombinedCheckContradictions(), trainset=trainset, valset=devset)

Going to sample between 1 and 4 traces per predictor.
Will attempt to train 16 candidate sets.
3 2 False
1 2 False
0 3 False
3 2 False
0 3 False


Average Metric: 12 / 20  (60.0): 100%|██████████| 20/20 [00:00<00:00, 574.44it/s]
/home/lbartolome/.local/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:137: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


1 1 True
0 2 False
0 3 False
0 0 True
0 0 True
1 2 False
1 1 True
0 0 True
1 1 True
0 0 True
0 0 True
0 0 True
0 0 True
0 0 True
0 0 True
Average Metric: 12 / 20  (60.0%)
Score: 60.0 for set: [0, 0]
New best score: 60.0 for seed -3
Scores so far: [60.0]
Best score: 60.0
0 2 False
0 3 False


  0%|          | 0/20 [00:00<?, ?it/s]

1 1 True
3 2 False
1 2 False
1 1 True
0 3 False
1 2 False
1 1 True
0 0 True
0 0 True


...

1 2 False
1 2 False
1 1 True
0 0 True
0 0 True
1 1 True
0 0 True
0 0 True


Average Metric: 0 / 7  (0.0):  30%|███       | 6/20 [00:00<00:00, 133.15it/s]

0 0 True
0 0 True
0 0 True
0 0 True
0 0 True


Average Metric: 11 / 20  (55.0): 100%|██████████| 20/20 [00:00<00:00, 332.65it/s]

Average Metric: 11 / 20  (55.0%)
Score: 55.0 for set: [4, 4]
Scores so far: [60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 55.0, 60.0, 60.0, 60.0, 55.0, 60.0, 60.0, 60.0, 60.0, 55.0]
Best score: 60.0
Average of max per entry across top 1 scores: 0.6
Average of max per entry across top 2 scores: 0.6
Average of max per entry across top 3 scores: 0.6
Average of max per entry across top 5 scores: 0.6
Average of max per entry across top 8 scores: 0.6
Average of max per entry across top 9999 scores: 0.6
19 candidate programs found.


In [16]:
llm.history[-1]

{'prompt': 'Identify if there are contradictions with the provided context.\n\n---\n\nFollow the following format.\n\nContext: facts here are assumed to be true\n\nText: ${text}\n\nFaithfulness: True/False indicating if text contradicts the context\n\nRationale: Why the new text contradicts the context or not\n\n---\n\nContext: With one important exception. The exception was the shot. Compared to women using a hormonal IUD, women using the shot were three times more likely to have a blood clot.\t\nText: Careful, not all methods are safe for new moms to use: As if they didn’t have their hands full already, women who have just given birth have to be especially careful when choosing a contraceptive. The CDC has just updated its guidelines on which methods are safe for new moms to use, and when. While hormonal methods that don’t contain estrogen—the shot, the Mirena IUD, the implant and the mini-pill—are safe for women to use immediately after giving birth, combination pills, which contain

In [17]:
contradiction_checker(context=devset[0].context, text=devset[0].text)

Prediction(
    Faithfulness=False,
    rationale='The text states that "Around 15% of pregnancies end in miscarriage," which contradicts the context that says "Pregnancy loss occurs in up to 1 in every 4 pregnancies" (which is 25%). Additionally, the text mentions that "everyone’s risk of miscarriage declines each week of pregnancy if the pregnant person has no other health conditions," which is not supported by the context provided. The context does not specify that the risk declines each week or that it is contingent on the absence of other health conditions.. Reasoning: Let\'s think step by step in order to identify any contradictions or discrepancies. \n\n1. The context states that pregnancy loss occurs in up to 1 in every 4 pregnancies, which translates to a 25% risk.\n2. The text mentions that around 15% of pregnancies end in miscarriage, which is lower than the 25% mentioned in the context.\n3. The context states that most pregnancy losses happen during the first trimester, whi

## Loading info from model

In [18]:
#!pip install pyarrow --upgrade

In [19]:
path_orig_en = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/corpus_pass_en_tr.parquet")
path_orig_es = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/corpus_pass_es_tr.parquet")
path_source = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/source/corpus_rosie/passages/translated_stops_filtered_by_al/df_1.parquet")

path_model = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/data/models/LDA_FILTERED_AL/rosie_1_20")

path_corpus_en = path_model / "train_data" / "corpus_EN.txt"
path_corpus_es = path_model / "train_data" / "corpus_ES.txt"

persist_directory = (path_model / 'db_contr_mono').as_posix()

In [20]:
raw = pd.read_parquet(path_source)
raw.head()

,id_preproc,lemmas,lemmas_tr,doc_id,text,text_tr,lang
0,0,implant_port information technology easy child prolonged treatment blood draw certain chronic illness place port noticeable information technology interfere_daily activity healthcare_provider discuss pro_con implant_port,puerto_implantado facilitar hijo recibir tratamiento prolongado extracción sangre enfermedad crónico puerto perceptible interferir mayoría actividad diario proveedor_atención médico discutir_pro puerto_implantado,EN_657272_107210-27,"Implanted ports make it easier for you or your child to get prolonged treatments or blood draws for certain chronic illnesses. Once in place, a port is barely noticeable. It shouldn’t interfere with most daily activities. Your healthcare provider can discuss the pros and cons of using an implanted port.","Los puertos implantados facilitan que usted o su hijo reciban tratamientos prolongados o extracciones de sangre para ciertas enfermedades crónicas. Una vez en su lugar, un puerto es apenas perceptible. No debe interferir con la mayoría de las actividades diarias. Su proveedor de atención médica puede discutir los pros y los contras del uso de un puerto implantado.",EN
1,1,life_expectancy birth reach record high year total population disparity life_expectancy birth remain race sex population population increase life_expectancy past_decade additional information life_expectancy available,esperanza_vida nacer alcanzar_máximo precedente año población unidos disparidad esperanza_vida nacer mantener población raza sexo población experimentar aumento esperanza_vida decenio obtener él esperanza_vida,EN_403516_60988-2,"Life expectancy at birth reached a record high of 77.9 years in 2005 for the total U.S. population. Disparities in life expectancy at birth remain among the race/sex populations, although all populations have had increases in life expectancy during the past decade. Additional information regarding life expectancy is available at http://www.cdc.gov/nchs/deaths.htm.","La esperanza de vida al nacer alcanzó un máximo sin precedentes de 77,9 años en 2005 para la población total de los Estados Unidos. Las disparidades en la esperanza de vida al nacer se mantienen entre las poblaciones de raza y sexo, aunque todas las poblaciones han experimentado un aumento de la esperanza de vida durante el último decenio. Puede obtenerse más información sobre la esperanza de vida en http://www.cdc.gov/nchs/deaths.htm.",EN
2,2,shazam_hussain arterial common location carotid_artery originate heart brain,shazam_hussain arterial común provenir arteria_carótida originar él corazón llegar cerebro,EN_736110_112440-10,"Shazam Hussain: Usually, on the arterial side. The most common locations, they either come from the carotid arteries, or they can originate in the heart and then come up to the brain.","Shazam Hussain: Por lo general, en el lado arterial. Los lugares más comunes, o provienen de las arterias carótidas, o pueden originarse en el corazón y luego llegar al cerebro.",EN
3,3,rifaximin cause effect doctor unusual_problem medication,rifaximina causar secundario_llamir médico problema_inusual tomar medicamento,EN_1279033_283895-10,Rifaximin may cause other side effects. Call your doctor if you have any unusual problems while taking this medication.,La rifaximina puede causar otros efectos secundarios. Llame a su médico si tiene algún problema inusual mientras toma este medicamento.,EN
4,4,point healing_process need dressing provider leave wound uncover,curación necesitar apósito proveedor dejar herida descubierto,EN_1241536_280479-12,"At some point during the healing process, you will not need a dressing anymore. Your provider will tell you when you can leave your wound uncovered.","En algún momento durante el proceso de curación, ya no necesitará un apósito. Su proveedor le dirá cuándo puede dejar la herida descubierta.",EN


In [21]:
with path_corpus_en.open("r", encoding="utf-8") as f:
    lines = [line for line in f.readlines()]
corpus_en = [line.rsplit(" 0 ")[1].strip().split() for line in lines]

ids = [line.split(" 0 ")[0] for line in lines]
df_en = pd.DataFrame({"lemmas": [" ".join(doc) for doc in corpus_en]})
df_en["doc_id"] = ids
df_en["len"] = df_en['lemmas'].apply(lambda x: len(x.split()))
df_en["id_top"] = range(len(df_en))

In [22]:
df_en_raw = df_en.merge(raw, how="inner", on="doc_id")[["doc_id", "id_top", "id_preproc", "lemmas_x", "text", "len"]]

# Read thetas 
thetas = sparse.load_npz(path_model.joinpath(f"mallet_output/{'EN'}/thetas.npz")).toarray()
betas = np.load((path_model.joinpath(f"mallet_output/{'EN'}/betas.npy")))
def get_thetas_str(row,thetas):
    return " ".join([f"{id_}|{round(el, 4)}" for id_,el in enumerate(thetas[row]) if el!=0.0])

def get_most_repr_tpc(row,thetas):
    return np.argmax(thetas[row])

# Save thetas in dataframe and "assigned topic"
df_en_raw["thetas"] = df_en_raw.apply(lambda row: get_thetas_str(row['id_top'], thetas), axis=1)
df_en_raw["id_tpc"] = df_en_raw.apply(lambda row: get_most_repr_tpc(row['id_top'], thetas), axis=1)
df_en_raw

,doc_id,id_top,id_preproc,lemmas_x,text,len,thetas,id_tpc
0,EN_657272_107210-27,0,0,implant_port information technology easy child prolonged treatment blood draw certain chronic illness place port noticeable information technology interfere_daily activity healthcare_provider discuss pro_con implant_port,"Implanted ports make it easier for you or your child to get prolonged treatments or blood draws for certain chronic illnesses. Once in place, a port is barely noticeable. It shouldn’t interfere with most daily activities. Your healthcare provider can discuss the pros and cons of using an implanted port.",23,2|0.4090999960899353 15|0.5909000039100647,15
1,EN_403516_60988-2,1,1,life_expectancy birth reach record high year total population disparity life_expectancy birth remain race sex population population increase life_expectancy past_decade additional information life_expectancy available,"Life expectancy at birth reached a record high of 77.9 years in 2005 for the total U.S. population. Disparities in life expectancy at birth remain among the race/sex populations, although all populations have had increases in life expectancy during the past decade. Additional information regarding life expectancy is available at http://www.cdc.gov/nchs/deaths.htm.",23,0|0.9060999751091003 17|0.04769999906420708 19|0.04619999974966049,0
2,EN_736110_112440-10,2,2,shazam_hussain arterial common location carotid_artery originate heart brain,"Shazam Hussain: Usually, on the arterial side. The most common locations, they either come from the carotid arteries, or they can originate in the heart and then come up to the brain.",8,1|0.0032999999821186066 2|0.8410999774932861 4|0.004600000102072954 7|0.12639999389648438 13|0.00430000014603138 14|0.004699999932199717 15|0.005900000222027302 17|0.006399999838322401 18|0.0031999999191612005,2
3,EN_1279033_283895-10,3,3,rifaximin cause effect doctor unusual_problem medication,Rifaximin may cause other side effects. Call your doctor if you have any unusual problems while taking this medication.,6,0|0.003800000064074993 1|0.004999999888241291 2|0.006099999882280827 3|0.004000000189989805 4|0.006899999920278788 5|0.003100000089034438 6|0.003700000001117587 7|0.010300000198185444 8|0.0031999999191612005 9|0.004000000189989805 10|0.003700000001117587 13|0.006500000134110451 14|0.0071000000461936 15|0.9099000096321106 16|0.004600000102072954 17|0.009700000286102295 18|0.004800000227987766 19|0.003599999938160181,15
4,EN_1241536_280479-12,4,4,point healing_process need dressing provider leave wound uncover,"At some point during the healing process, you will not need a dressing anymore. Your provider will tell you when you can leave your wound uncovered.",8,1|0.003800000064074993 2|0.004600000102072954 4|0.005200000014156103 7|0.007699999958276749 13|0.9524000287055969 14|0.0052999998442828655 15|0.006599999964237213 16|0.0034000000450760126 17|0.007300000172108412 18|0.003599999938160181,13
...,...,...,...,...,...,...,...,...
985220,EN_384204_58596-4,985220,1393108,october world health organization nrevss_laboratory test specimen influenza virus positive influenza virus influenza b virus influenza virus subtype influenza virus influenza virus,"Since October 2, 2005, WHO and NREVSS laboratories have tested 110,066 specimens for influenza viruses, of which 13,532 (12.3%) were positive. Of these, 11,989 (88.6%) were influenza A viruses, and 1,543 (11.4%) were influenza B viruses. Of the 11,989 influenza A viruses, 4,875 (40.7%) have been subtyped; 4,629 (95.0%) were influenza A (H3N2) viruses, and 246 (5.0%) were influenza A (H1N1) viruses.",22,11|0.8561999797821045 16|0.14380000531673431,11
985221,EN_1389456_301244-3,985221,1393109,pectus_carinatum diagnosed chest_wall deformity evident infancy develop obvious child grow abnormality note parent evident physician routine chest examination,"How is Pectus Carinatum Diagnosed: Chest wall deformities can be evident in infancy, but many also develop or become more obvio

In [23]:
# Load topic-keys
keys = []
with open((path_model / "mallet_output/EN" / "topickeys.txt"), 'r') as file:
    for line in file:
        # Strip leading and trailing whitespace
        stripped_line = line.strip()
        # Split the line into parts and ignore the first two parts (number and float)
        parts = stripped_line.split(maxsplit=2)
        if len(parts) > 2:
            text_part = parts[2]
            keys.append(text_part)
print(*[f"Topic {i}: {el}" for i, el in enumerate(keys)], sep='\n')

Topic 0: age year rate high woman prevalence report white increase adult group child black population person health non_hispanic state low man
Topic 1: baby pregnancy birth woman infant information technology health risk pregnant mother week world organization delivery control breastfeed newborn fetal care
Topic 2: heart surgery technology information procedure test blood lung image remove call body small perform tissue child surgeon doctor tube place
Topic 3: cancer treatment study patient cell tumor disease research therapy mayo_clinic treat surgery develop breast_cancer include researcher risk lung clinical clinical_trial
Topic 4: health risk world organization disease people condition diabetes high adult chronic heart factor increase age stroke child problem blood_pressure disorder
Topic 5: health patient exposure person care organization world infection worker risk facility contact case include day work transmission report control hospital
Topic 6: cell protein gene disorder condi

## VectorDatabase

Store one index per topic

In [148]:
tpc = 1
df_tpc = df_en_raw[df_en_raw.id_tpc == tpc]#.reset_index()[["doc_id", "text"]]
df_tpc

,doc_id,id_top,id_preproc,lemmas_x,text,len,thetas,id_tpc
45,EN_4908_321-60,45,65,chorioamnionitis condition pregnancy cause unexplained_fever uterine tenderness high white blood cell count rapid heart rate fetus rapid heart rate woman smell vaginal_discharge,"Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.",22,1|0.42800000309944153 5|0.04820000007748604 10|0.0957999974489212 18|0.42800000309944153,1
65,EN_245531_51078-8,65,91,q hear vitamin_k shot cause cancer true early small study england association vitamin_k shot childhood cancer association mean thing happen time person ultrasound cause figure vitamin_k cause childhood cancer important newborn expect vitamin_k shot vitamin_k cause cancer expect association group child scientist look association child association vitamin_k childhood cancer study,"Q: I heard that the vitamin K shot might cause cancer. Is this true: A: No. In the early 1990s, a small study in England found an “association” between the vitamin K shot and childhood cancer. An association means that two things are happening at the same time in the same person, but doesn’t tell us whether one causes the other. Figuring out whether vitamin K might cause childhood cancer was very important because every newborn is expected to get a vitamin K shot. If vitamin K was causing cancer, we would expect to see the same association in other groups of children. Scientists looked to see if they could find the same association in other children, but this association between vitamin K and childhood cancer was never found again in any other study.",50,1|0.3716000020503998 3|0.2556000053882599 9|0.16279999911785126 17|0.20990000665187836,1
71,EN_169761_4756-1,71,98,know sure want pregnant pregnant think tube tie permanent sterilization procedure talk doctor sterilization tricky young worry doctor talk information technology,"But what if you know for sure that you never want to be pregnant again, or even pregnant ever? You might be thinking about having your “tubes tied,” or going through a permanent sterilization procedure. But talking to your doctor about sterilization can be tricky! Especially if you’re young, you may worry that your doctor won’t take you seriously or will try to talk you out of it.",21,1|0.4088999927043915 7|0.0032999999821186066 15|0.2937999963760376 17|0.2939999997615814,1
122,EN_1390617_301348-44,122,171,urine test look substance certain microcephaly,Urine test. This is done to look for a substance that may show a certain type of microcephaly.,6,0|0.004600000102072954 1|0.6610000133514404 2|0.007400000002235174 3|0.004800000227987766 4|0.00839999970048666 5|0.003800000064074993 6|0.004399999976158142 7|0.012500000186264515 8|0.0038999998942017555 9|0.004900000058114529 10|0.0044999998062849045 11|0.0031999999191612005 12|0.003599999938160181 13|0.007899999618530273 14|0.00860000029206276 15|0.010700000450015068 16|0.005499999970197678 17|0.011800000444054604 18|0.2240999937057495 19|0.00430000014603138,1
167,EN_1400860_302313-6,167,241,child risk x_link agammaglobulinemia woman world health organization carrier chance pass faulty x_chromosome child true pregnancy daughter gene healthy carrier mother son gene x_link agammaglobulinemia,"Which children are at risk for X-linked agammaglobulinemia: Women who are carriers have a 1 in 2 chance of passing the faulty X chromosome to a child. This is true for every pregnancy. If a daughter gets the gene, she will likely be a healthy carrier like her mother. If a son gets the gene, he will have X-linked agammaglobulinemia.",25,1|0.5192999839782715 4|0.0414000004529953 6|0.439300000667572,1
...,...,...,...,...,...,...,...,...
985102,EN_870881_124186-24,985102,1392948,plan b step pill_progestin form emergency_contraception available counter ulipristal_ella prescription medication t

In [60]:
"""
model = SentenceTransformer("all-mpnet-base-v2", device="cuda")
embeds = model.encode(df_tpc["text"].tolist(), show_progress_bar=False)
dists = cosine_distances(embeds)
np.quantile(dists, (0.01, 0.05, 0.1))
"""

'\nmodel = SentenceTransformer("all-mpnet-base-v2", device="cuda")\nembeds = model.encode(df_tpc["text"].tolist(), show_progress_bar=False)\ndists = cosine_distances(embeds)\nnp.quantile(dists, (0.01, 0.05, 0.1))\n'

## Filtering 

In [81]:
import re

patterns = [
    r'\blife expectancy\b',
    r'\bmortality\b',
    r'\bprevalence\b',
    r'\bstatistical\b',
    r'\bQuickStats\b',
    r'\brate\b',
    r'\baverage\b',
    r'\bincreased\b',
    r'\bdecreased\b',
    r'\bgains\b',
    r'\bdecline\b',
    r'\bpercentage\b',
    r'\btrend\b',
    r'\bU\.S\. population\b',
    r'\bamong U\.S\. Adults\b',
    r'\bHispanic population\b',
    r'\d{1,2}%|\d{1,2}-\d{1,2}',  # Matches percentages and age ranges
    r'\d+\.\d+|\(\d+\.\d+-\d+\.\d+\)',  # Matches decimal numbers and ranges in parentheses
    #r'\b[A-Z][a-z]*\b',  # Matches capitalized words (e.g., "Education", "Income")
    r'\b(\d{4})\b',  # Matches years (4-digit numbers)
    r'Age \(yrs\)|Education|Income|Region',  # Specific keywords related to demographic data
]

def is_too_specific(text):
    for pattern in patterns:
        if re.search(pattern, text, re.IGNORECASE):
            return True
    return False

#df_filtered = df_tpc[~df_tpc['text'].apply(is_too_specific)]

df_filtered = df_tpc
df_filtered

,doc_id,text
0,EN_4908_321-60,"Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge."
1,EN_245531_51078-8,"Q: I heard that the vitamin K shot might cause cancer. Is this true: A: No. In the early 1990s, a small study in England found an “association” between the vitamin K shot and childhood cancer. An association means that two things are happening at the same time in the same person, but doesn’t tell us whether one causes the other. Figuring out whether vitamin K might cause childhood cancer was very important because every newborn is expected to get a vitamin K shot. If vitamin K was causing cancer, we would expect to see the same association in other groups of children. Scientists looked to see if they could find the same association in other children, but this association between vitamin K and childhood cancer was never found again in any other study."
2,EN_169761_4756-1,"But what if you know for sure that you never want to be pregnant again, or even pregnant ever? You might be thinking about having your “tubes tied,” or going through a permanent sterilization procedure. But talking to your doctor about sterilization can be tricky! Especially if you’re young, you may worry that your doctor won’t take you seriously or will try to talk you out of it."
3,EN_1390617_301348-44,Urine test. This is done to look for a substance that may show a certain type of microcephaly.
4,EN_1400860_302313-6,"Which children are at risk for X-linked agammaglobulinemia: Women who are carriers have a 1 in 2 chance of passing the faulty X chromosome to a child. This is true for every pregnancy. If a daughter gets the gene, she will likely be a healthy carrier like her mother. If a son gets the gene, he will have X-linked agammaglobulinemia."
...,...,...
39291,EN_870881_124186-24,"The Plan B One-Step® pill and other progestin-only forms of emergency contraception are available over the counter. However, ulipristal (ella®) is a prescription-only medication. You should also talk to your healthcare provider before taking combined birth control pills to make sure you take the correct dosage (amount)."
39292,EN_526170_87128-3,"Breast changes continue to occur over the lifespan, with lobes, or small subdivisions of breast tissue, developing first. Mammary glands develop next, and consist of 15-24 lobes. Mammary glands are influenced by hormones activated in puberty. Involution or shrinkage of the milk ducts is the final major change that occurs within the breast tissue. A gradual shrinking of the mammary glands (involution) typically begins around the age of 35."
39293,EN_840478_120635-6,"Precocious puberty can have different causes, depending on the sex of the child, age of onset, and other factors. In most cases, the causes of central precocious puberty in girls are unknown, but it can be treated effectively."
39294,EN_1369827_298042-7,The implant. IUDs. The shot. Learn more about available birth control methods.


In [ ]:
# 39296 vs 32229

In [82]:
import faiss

def create_faiss_index(df, text_column, id_column, model_name="all-mpnet-base-v2", index_file="faiss_index.index"):
    """
    Create a FAISS index from a DataFrame containing text data.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    text_column (str): The name of the column containing text data.
    id_column (str): The name of the column containing unique identifiers for the texts.
    model_name (str): The name of the SentenceTransformer model to use for embeddings.
    index_file (str): The file path to save the FAISS index.

    Returns:
    index: The FAISS index object.
    model: The SentenceTransformer model used for embeddings.
    ids: List of document IDs.
    texts: List of document texts.
    """
    # Extract texts and ids
    texts = df[text_column].tolist()
    ids = df[id_column].tolist()

    # Load pre-trained SentenceTransformer model
    model = SentenceTransformer(model_name, device="cuda")

    # Calculate embeddings for the texts
    embeddings = model.encode(texts, show_progress_bar=False)

    # Create a FAISS index
    d = embeddings.shape[1]
    index = faiss.IndexFlatIP(d)  # Using Inner Product to approximate cosine similarity

    # Normalize the embeddings to unit length for cosine similarity
    faiss.normalize_L2(embeddings)

    # Add embeddings to the index
    index.add(embeddings)

    # Save the index to a file
    faiss.write_index(index, index_file)

    return index, model, ids, texts

def retrieve_similar_documents(query_text, model, index, ids, texts, k=5):
    """
    Retrieve the k most similar documents to the query text.

    Parameters:
    query_text (str): The query text.
    model: The SentenceTransformer model used for embeddings.
    index: The FAISS index object.
    ids (list): List of document IDs.
    texts (list): List of document texts.
    k (int): The number of nearest neighbors to retrieve.

    Returns:
    list: A list of dictionaries containing document IDs, distances, and texts of the k most similar documents.
    """
    # Encode the query text
    query_embedding = model.encode([query_text], show_progress_bar=False)
    faiss.normalize_L2(query_embedding)
    
    # Search the index for the k nearest neighbors
    distances, indices = index.search(query_embedding, k)
    
    # Retrieve the corresponding texts and ids
    results = []
    for i in range(k):
        result = {
            "document_id": ids[indices[0][i]],
            "distance": distances[0][i],
            "text": texts[indices[0][i]]
        }
        results.append(result)
    
    return results


index_en, model_en, ids_en, texts_en = create_faiss_index(df_filtered, text_column='text', id_column='doc_id', index_file='faiss_index_en.index')

/home/lbartolome/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [83]:
doc = df_filtered.iloc[0]
doc.text

'Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.'

In [144]:
betas

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.61951467e-02, 1.36294639e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.48771955e-05, 1.56591284e-02, 1.60649821e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 5.37939900e-02, 4.95731510e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.01764297e-02, 1.76429706e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.61320700e-03, 1.30140291e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [145]:
# Load vocab dictionaries
vocab_w2id = {}
vocab_id2w = {}

# Open the file and read the lines
with open((path_model / "mallet_output/EN" / "vocab_freq.txt"), 'r', encoding='utf8') as file:
    for i, line in enumerate(file):
        # Strip leading and trailing whitespace
        stripped_line = line.strip()
        # Split the line into words and numbers
        parts = stripped_line.split()
        if parts:
            # Get the word (first part)
            wd = parts[0]
            # Populate the dictionaries
            vocab_w2id[wd] = i
            vocab_id2w[str(i)] = wd

In [146]:
vocab_w2id["chorioamnionitis"]

670

In [147]:
betas[1,620]

0.00026411439926551044

In [159]:
vocab_w2id["condition"]

233

In [167]:
aux = df_tpc[df_tpc.doc_id.isin(sim_docs)].copy()

def get_betas_prop(row, tpc=1):
    els = row.split()
    betas_el = [f"{vocab_w2id[el]}|{betas[tpc, vocab_w2id[el]]}" for el in els if el in vocab_w2id.keys()]
    return betas_el

def count_words_in_tpc(row, tpc=1):
    els = row.split()
    return sum([betas[tpc, vocab_w2id[el]] for el in els if el in vocab_w2id.keys()])
aux["betas_doc"] = aux["lemmas_x"].apply(get_betas_prop)
aux["sum"] =  aux["lemmas_x"].apply(count_words_in_tpc)
aux

,doc_id,id_top,id_preproc,lemmas_x,text,len,thetas,id_tpc,betas_doc,sum
45,EN_4908_321-60,45,65,chorioamnionitis condition pregnancy cause unexplained_fever uterine tenderness high white blood cell count rapid heart rate fetus rapid heart rate woman smell vaginal_discharge,"Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.",22,1|0.42800000309944153 5|0.04820000007748604 10|0.0957999974489212 18|0.42800000309944153,1,"[670|6.481928480142564e-05, 233|0.0033725377136980565, 527|0.03980871538819496, 671|0.0, 672|0.0006201367396673706, 673|4.0632984502386224e-05, 23|0.003446064066607136, 674|0.0, 7|0.0003773062846650149, 165|0.0, 675|0.00010835462533969658, 676|7.06239968731951e-05, 40|0.00015866212996169858, 581|0.0012276966031792408, 539|0.005359684146267135, 676|7.06239968731951e-05, 40|0.00015866212996169858, 581|0.0012276966031792408, 532|0.024025703265053794, 677|0.0, 678|0.00019542530641623849]",0.080333
89747,EN_738076_112593-0,89747,126809,chorioamnionitis intra_amniotic infection infection affect person pregnancy information technology membrane_surround fetus uterus chorion amnion,Chorioamnionitis (intra-amniotic infection) is a serious infection that affects a person during pregnancy. It gets its name from the two membranes that surround a fetus in your uterus: the chorion and the amnion.,14,1|0.8454999923706055 4|0.14329999685287476 7|0.004000000189989805 15|0.0034000000450760126 17|0.003800000064074993,1,"[670|6.481928480142564e-05, 95671|7.739616095692614e-06, 543|0.002073249661633659, 543|0.002073249661633659, 528|0.0030474738376789663, 209|0.0, 527|0.03980871538819496, 1|0.016195146680236795, 2|0.013629463944514692, 24038|1.0641972131577343e-05, 539|0.005359684146267135, 5029|0.0037421043822673788, 45436|1.0641972131577343e-05, 39772|1.741413621530838e-05]",0.086040
115916,EN_1356177_296676-164,115916,163902,chorioamnionitis_chorioamnionitis chor y nee ny infection placenta_amniotic fluid woman information technology information technology common preterm_labor delivery,Chorioamnionitis: Chorioamnionitis [chor-y-oh-am-nee-oh-NY-tis] is an infection of the placenta and the amniotic fluid. Only a few women get it. But it is a common cause of preterm labor and delivery.,16,1|0.6601999998092651 7|0.003800000064074993 15|0.0031999999191612005 17|0.3328000009059906,1,"[38778|6.7721640837310364e-06, 108943|9.674520119615767e-07, 57040|1.9349040239231534e-06, 13975|0.0, 543|0.002073249661633659, 59365|3.386082041865518e-05, 1401|0.000315389355899474, 532|0.024025703265053794, 1|0.016195146680236795, 2|0.013629463944514692, 1|0.016195146680236795, 2|0.013629463944514692, 36|0.002674037361061798, 6178|0.0004895307180525578, 534|0.006997580402518084]",0.096268
173216,EN_289067_53287-52,173216,244917,association observe epidural labor analgesia fever chorioamnionitis overdiagnose woman epidural lead unnecessary diagnostic evaluation unnecessary exposure empirical antibiotic neonate multistate surveillance datum suggest epidural common birth intrapartum temperature birth physician diagnosis chorioamnionitis birth remain rare cdc_unpublished datum consultation obstetric provider determine chorioamnionitis suspect important guide neonatal management,"Because an association has been observed between epidural labor analgesia and fever, chorioamnionitis might be overdiagnosed in women with epidurals, which could lead to unnecessary diagnostic evaluations and unnecessary exposure to empirical antibiotics in neonates (216). However, multistate surveillance data suggest that although epidural use is common (in 67% of births), intrapartum temperature of ≥100.4ºF [≥38.0ºC] (3.3% of births) and physician diagnoses of chorioamnionitis (3.1% of births) remain relatively rare (102) (CDC, unpublished data, 2009). Consultation with obstetric providers to determ

In [149]:
sim_docs = [doc['document_id'] for doc in retrieve_similar_documents(doc.text, model_en, index_en, ids_en, texts_en, k=20)]
sim_docs

['EN_4908_321-60',
 'EN_2821_223-137',
 'EN_641141_105854-0',
 'EN_738076_112593-0',
 'EN_1396726_301900-34',
 'EN_1396716_301900-0',
 'EN_1356177_296676-164',
 'EN_1396725_301900-33',
 'EN_641144_105854-5',
 'EN_289066_53287-51',
 'EN_641152_105854-20',
 'EN_1396729_301900-37',
 'EN_738077_112593-3',
 'EN_289067_53287-52',
 'EN_1394388_301670-14',
 'EN_1396761_301904-0',
 'EN_310405_53660-196',
 'EN_332560_8920-3',
 'EN_1067827_191618-43',
 'EN_332565_8920-9']

In [173]:
retrieve_similar_documents("What is chorioamnionitis?", model_en, index_en, ids_en, texts_en, k=20)

[{'document_id': 'EN_1396726_301900-34',
  'distance': 0.85260457,
  'text': 'Chorioamnionitis is an infection of the placenta and the amniotic fluid.'},
 {'document_id': 'EN_641141_105854-0',
  'distance': 0.84959567,
  'text': 'What is chorioamnionitis: Chorioamnionitis (intra-amniotic infection) is a serious infection that affects a person during pregnancy. It gets its name from the two membranes that surround a fetus in your uterus: the chorion and the amnion.'},
 {'document_id': 'EN_738076_112593-0',
  'distance': 0.8263569,
  'text': 'Chorioamnionitis (intra-amniotic infection) is a serious infection that affects a person during pregnancy. It gets its name from the two membranes that surround a fetus in your uterus: the chorion and the amnion.'},
 {'document_id': 'EN_1396716_301900-0',
  'distance': 0.79133976,
  'text': "What is chorioamnionitis: Chorioamnionitis is an infection of the placenta and the amniotic fluid. Only a few women get it, but, it's a common cause of preterm 

In [123]:
k = 10

id_to_index = {}
no_contrads = []
contradictions = []
free_of_contradictions = []

def add_doc_to_no_contrads(doc):
    id_index = len(no_contrads)
    id_to_index[id_index] = doc.doc_id
    no_contrads.append(doc.doc_id)

# Add initial document
doc_0 = df_tpc.iloc[0]
add_doc_to_no_contrads(doc_0)

def parse_response(resp):
    if "0" in resp.answer:
        return 0
    elif "1" in resp.answer:
        return 1
    elif "2" in resp.answer:
        return 2
    elif "3" in resp.answer:
        return 3
    else:
        print(f"No valid answer from Contradiction Checker: {resp}")

def get_closest_indices(embeddings, indices, k):
    mask = np.ones_like(embeddings, dtype=bool)
    mask[indices] = False
    embeddings[mask] = 100

    # Adding the similarity threshold mask
    similarity_threshold = 0.1
    similarity_mask = embeddings >= similarity_threshold
    embeddings[~similarity_mask] = 100

    closest_indices = np.argsort(embeddings)[:k]
    print(embeddings[closest_indices])
    return [idx for idx in closest_indices if embeddings[idx] != 100]

def get_context_data(doc_id, df):
    context_data = df[df.doc_id == doc_id].iloc[0].to_dict()
    context_data["context"] = context_data.pop("text")
    return context_data

for index, row in df_filtered[:10].iterrows():#.sample(n=20, random_state=1)
    if index % 100 == 0:
        print(f"-- -- Processing index {index}")
    #this_emb = dists.copy()[index]
    #indices = list(id_to_index.keys())
    #closest_indices = get_closest_indices(this_emb, indices, k)
    
    similar_docs = retrieve_similar_documents(row.text, model_en, index_en, ids_en, texts_en, k)

    responses = []
    for el in similar_docs:
        if el["text"] != row.text:
            doc_id = el["document_id"]
            sim = el["distance"]
            context_data = get_context_data(doc_id, df_filtered)
            text_data = get_context_data(row.doc_id, df_filtered)

            context = dspy.Example(**context_data).with_inputs('context', 'text')
            text = dspy.Example(**text_data).with_inputs('context', 'text')

            resp = contradiction_checker(context=context.context, text=text.context)
            resp_int = resp.answer
            if resp_int is None:
                print("No valid answer from Contradiction Checker")
                continue

            responses.append(resp_int)

            if resp_int != "0":
                contradictions.append([doc_id, row.doc_id, context_data["context"], text_data["context"], resp.rationale, resp_int, sim])
            else:
                free_of_contradictions.append([doc_id, row.doc_id, context_data["context"], text_data["context"], resp.rationale, resp_int, sim])
            
    if len(set(responses)) <= 1:
        no_contrads.append(row.doc_id)
        id_to_index[index] = row.doc_id

print(no_contrads)

-- -- Processing index 0
['EN_4908_321-60', 'EN_4908_321-60', 'EN_245531_51078-8', 'EN_169761_4756-1', 'EN_932892_155275-3']


In [124]:
cont_df = pd.DataFrame(contradictions, columns=["docid1", "docid2", "text1", "text2", "rationale", "contr", "distance"])
cont_df

,docid1,docid2,text1,text2,rationale,contr,distance
0,EN_1357558_296737-31,EN_1390617_301348-44,Amniocentesis. This tests the fluid from the sac surrounding the baby (amniotic fluid).,Urine test. This is done to look for a substance that may show a certain type of microcephaly.,"The context specifies that amniocentesis tests the fluid from the sac surrounding the baby (amniotic fluid). The text, however, describes a urine test, which is a different type of test and not related to amniotic fluid. Therefore, the text contradicts the context.. produce the LabelInt. We start by identifying the main elements in the context and the text. The context describes amniocentesis, which involves testing the amniotic fluid surrounding the baby. The text, however, describes a urine test, which is a different type of test altogether. The two tests are distinct in their methods and the substances they analyze. Therefore, there is a discrepancy between the context and the text.",2,0.690844
1,EN_1388313_301137-40,EN_1390617_301348-44,Testing for amniotic fluid. This lets your provider know if the sac around the baby has broken.,Urine test. This is done to look for a substance that may show a certain type of microcephaly.,"The context specifies that testing for amniotic fluid is done to determine if the sac around the baby has broken. The text, however, describes a urine test used to look for a substance that may indicate a certain type of microcephaly. These are two different tests with different purposes, so the text contradicts the context.. Reasoning: Let's think step by step in order to identify the type of contradiction. We start by noting that the context is about testing for amniotic fluid to determine if the sac around the baby has broken. The text, however, describes a urine test that looks for a substance indicating a certain type of microcephaly. These two tests serve entirely different purposes: one is for detecting amniotic fluid leakage, and the other is for diagnosing a specific condition in the baby. Therefore, the text does not align with the context provided.",1,0.690291
2,EN_992140_17643-14,EN_1390617_301348-44,"Microcephaly can be determined by measuring head circumference (HC) after birth. Although head circumference measurements may be influenced by molding and other factors related to delivery, the measurements should be taken on the first day of life because commonly-used birth head circumference reference charts by age and sex are based on measurements taken before 24 hours of age. The most important factor is that the head circumference is carefully measured and documented. If measurement within the first 24 hours of life is not done, the head circumference should be measured as soon as possible after birth. If the healthcare provider suspects the baby has microcephaly, he or she can request one or more tests to help confirm the diagnosis. For example, special tests like like magnetic resonance imaging can provide critical information on the structure of the baby’s brain that can help determine if the newborn baby had an infection during pregnancy. They also can help the healthcare provider look for other problems that might be present.",Urine test. This is done to look for a substance that may show a certain type of microcephaly.,"The context does not mention urine tests as a method for diagnosing microcephaly. It specifically mentions measuring head circumference and using special tests like magnetic resonance imaging to confirm the diagnosis and look for other problems. Therefore, the text about a urine test contradicts the provided context.. produce the LabelInt. We need to compare the information provided in the context with the statement in the text. The context specifies that head circumference measurements and imaging tests like MRI are used to diagnose microcephaly. It does not mention urine tests as a method for diagnosing microcephaly. The text introduces a urine test as a method to look for a substance that 

In [ ]:
cont_df.text1.values

In [121]:
non_cont_df = pd.DataFrame(free_of_contradictions, columns=["docid1", "docid2", "text1", "text2", "rationale", "contr", "distance"])
#non_cont_df.to_excel("contrs.xlsx")
non_cont_df

,docid1,docid2,text1,text2,rationale,contr,distance
0,EN_2821_223-137,EN_4908_321-60,"Chorioamnionitis - A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.","Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.","The text accurately describes chorioamnionitis as a condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge, which is consistent with the provided context.. The text accurately describes chorioamnionitis as a condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge, which is consistent with the provided context.",0,0.995141
1,EN_641141_105854-0,EN_4908_321-60,What is chorioamnionitis: Chorioamnionitis (intra-amniotic infection) is a serious infection that affects a person during pregnancy. It gets its name from the two membranes that surround a fetus in your uterus: the chorion and the amnion.,"Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.","The text accurately describes chorioamnionitis as a condition during pregnancy and lists symptoms that are consistent with the infection, such as unexplained fever, uterine tenderness, high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and foul-smelling vaginal discharge. There is no contradiction with the provided context.. The text accurately describes chorioamnionitis as a condition during pregnancy and lists symptoms that are consistent with the infection, such as unexplained fever, uterine tenderness, high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and foul-smelling vaginal discharge. There is no contradiction with the provided context.",0,0.764890
2,EN_738076_112593-0,EN_4908_321-60,Chorioamnionitis (intra-amniotic infection) is a serious infection that affects a person during pregnancy. It gets its name from the two membranes that surround a fetus in your uterus: the chorion and the amnion.,"Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.","The text describes symptoms and signs associated with chorioamnionitis, which is consistent with the context that identifies it as a serious infection during pregnancy. There is no contradiction between the context and the text.. The text describes symptoms and signs associated with chorioamnionitis, which is consistent with the context that identifies it as a serious infection during pregnancy. There is no contradiction between the context and the text.",0,0.755708
3,EN_1396726_301900-34,EN_4908_321-60,Chorioamnionitis is an infection of the placenta and the amniotic fluid.,"Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.","The text does not contradict the context. The context defines chorioamnionitis as an infection of the placenta and the amniotic fluid, while the text provides additional details about the symptoms and effects of 

In [142]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Define a function to extract the core concept
def extract_core_concept(text):
    doc = nlp(text)
    for token in doc:
        if token.dep_ == "nsubj" or token.dep_ == "ROOT":
            return token.text
        
def extract_important_concepts(text):
    doc = nlp(text)
    important_concepts = []
    for token in doc:
        if token.pos_ in ['NOUN'] and not token.is_stop:
            important_concepts.append(token.text)
    return important_concepts

# Examples
sentence1 = "Amniocentesis. This tests the fluid from the sac surrounding the baby (amniotic fluid)"
sentence2 = "Urine test. This is done to look for a substance that may show a certain type of microcephaly"

print(extract_important_concepts(sentence1))  # Output: Amniocentesis
print(extract_important_concepts(sentence2))  # Output: Urine test


['Amniocentesis', 'fluid', 'sac', 'baby', 'fluid']
['test', 'substance', 'type', 'microcephaly']


In [143]:
print(doc.text)
for el in retrieve_similar_documents(doc.text, model_en, index_en, ids_en, texts_en, k=20):
    print(extract_important_concepts(el["text"]), f"SIM: {el['distance']}")

Chorioamnionitis: A condition during pregnancy that can cause unexplained fever with uterine tenderness, a high white blood cell count, rapid heart rate in the fetus, rapid heart rate in the woman, and/or foul-smelling vaginal discharge.
['Chorioamnionitis', 'condition', 'pregnancy', 'fever', 'tenderness', 'blood', 'cell', 'count', 'heart', 'rate', 'fetus', 'heart', 'rate', 'woman', 'discharge'] SIM: 1.0
['condition', 'pregnancy', 'fever', 'tenderness', 'blood', 'cell', 'count', 'heart', 'rate', 'fetus', 'heart', 'rate', 'woman', 'discharge'] SIM: 0.9951412081718445
['chorioamnionitis', 'infection', 'infection', 'person', 'pregnancy', 'membranes', 'fetus', 'uterus', 'chorion', 'amnion'] SIM: 0.7648895382881165
['infection', 'infection', 'person', 'pregnancy', 'membranes', 'fetus', 'uterus', 'chorion', 'amnion'] SIM: 0.7557077407836914
['infection', 'placenta', 'fluid'] SIM: 0.7368187308311462
['chorioamnionitis', 'infection', 'placenta', 'fluid', 'women', 'cause', 'preterm', 'labor', '

In [128]:
test1= "With one important exception. The exception was the shot. Compared to women using a hormonal IUD, women using the shot were three times more likely to have a blood clot.\t"
test2="Careful, not all methods are safe for new moms to use: As if they didn’t have their hands full already, women who have just given birth have to be especially careful when choosing a contraceptive. The CDC has just updated its guidelines on which methods are safe for new moms to use, and when. While hormonal methods that don’t contain estrogen—the shot, the Mirena IUD, the implant and the mini-pill—are safe for women to use immediately after giving birth, combination pills, which contain estrogen, shouldn’t be used until three weeks after giving birth, since estrogen can increase a woman’s risk of blood clots.\t"

In [139]:
print(extract_important_concepts(test1))
print(extract_important_concepts(test2))

['exception', 'exception', 'shot', 'women', 'IUD', 'women', 'shot', 'times', 'blood', 'clot']
['methods', 'moms', 'hands', 'women', 'birth', 'contraceptive', 'CDC', 'guidelines', 'methods', 'moms', 'methods', 'estrogen', 'shot', 'Mirena', 'IUD', 'implant', 'mini', '-', 'pill', 'women', 'birth', 'combination', 'pills', 'estrogen', 'weeks', 'birth', 'estrogen', 'woman', 'risk', 'blood', 'clots']
